In [1]:
import os
from datetime import datetime
from multiprocessing import Process

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
"""
Google Data Informal Documentation
"""

# Read in Google Mobility Data
google_data = pd.read_csv('./data/Google_Global_Mobility_Report.csv',
                          low_memory=False)

# Read in Apple Mobility Data
apple_data = pd.read_csv('./data/applemobilitytrends-2020-09-21.csv',
                         low_memory=False)


# Extract column names to be renamed
apple_date_columns = apple_data.loc[:, '1/13/2020':]
column_names = apple_date_columns.columns
updated_column_names = []

# Convert column names to have matching date format
for name in column_names:
    date = datetime.strptime(name, '%m/%d/%Y').strftime('%Y-%m-%d')
    updated_column_names.append(date)

# Update names and reform original DataFrame
apple_date_columns.columns = updated_column_names
apple_data = pd.concat([apple_data.loc[:, :'country'], apple_date_columns],
                       axis=1)

# Forcibly clean up duplicate date columns to preserve memory
del apple_date_columns
del column_names
del updated_column_names

# Break the data into more specific subsets.
# The data has the following structure (from broad to specific):
# Country/Region -> Sub-Region(States in the US) -> County -> City
apple_countries = apple_data.loc[apple_data['geo_type'] == 'country/region']
apple_sub_regions = apple_data.loc[apple_data['geo_type'] == 'sub-region']
apple_counties = apple_data.loc[apple_data['geo_type'] == 'county']
apple_cities = apple_data.loc[apple_data['geo_type'] == 'city']

In [3]:
google_data
google_data = google_data.fillna(0)

In [ ]:
# Extract Google country data into dataframe
for country in set(google_data['country_region'].to_list()):        
    
    # Second line ensures that no duplicate city data from countries is picked up
    country_data = google_data.loc[google_data['country_region'] == country]
    country_data = country_data.loc[country_data['sub_region_1'] == 0]

    # Seperates description information from mobility data
    # temp: stores mobility data
    # country_data: stores description information
    temp = country_data.transpose().iloc[7:]
    country_data = country_data.transpose().iloc[:7]
    country_data = country_data.iloc[:,:6]
    country_data = country_data.transpose()

    # creates a single column dataframe
    # will be used to label dataframe within country dataframe
    datatypes = temp.index.values.tolist()
    datatypes = pd.DataFrame(datatypes, columns=['datatype'])

    # renames column index in temp to use date format
    # renames row indices to be numeric
    # this makes concatenation work later
    temp.columns = temp.iloc[0]
    temp = temp.drop(temp.index[0])
    temp.index = list(range(6))
    datatypes = datatypes.drop(datatypes.index[0])
    datatypes.index = list(range(6))

    # creates country dataframe with all information
    # additional logic is needed to match overall column index format
    google_country_df = pd.concat([country_data, datatypes, temp], axis=1)
    google_country_df.rename(columns={'country_region_code':'geo_type',
                                      'country_region':'region',
                                      'sub_region_1':'sub-region',
                                      'sub_region_2':'country'}, inplace=True)

    # reorder columns to match country_df
    cols = list(google_country_df.columns.values)
    cols_reorder = ['geo_type',
                    'region',
                    'datatype',
                    'sub-region',
                    'country']
    cols = cols_reorder + cols[8:]
    
    google_country_df = google_country_df[cols].iloc[0:6]
    
    # Fill NaN with 0
    google_country_df = google_country_df.fillna(0)
    
    
    df = google_country_df.iloc[:, 5:]
    
    df = pd.concat([google_country_df.iloc[:, 0:6],
                                   df.groupby(df.columns, axis=1).mean()],
                                  axis=1)
    
    google_country_df = df.loc[:,~df.columns.duplicated()]
    
    # find matching country in country_df_list
    # append google data to matching dataframe
    for index, country_df in enumerate(country_df_list):
        if country_df['region'].iloc[0].strip() == country.strip():
            df = pd.concat([country_df, google_country_df], axis=0)
            country_df_list[index] = df.fillna(0)

In [27]:
country_df_list[2]

,geo_type,region,datatype,sub-region,country,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,...,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21
0,country/region,Australia,driving,NaN,NaN,100,102.98,104.21,108.63,109.08,...,83.67,85.82,89.24,93.18,95.04,101.92,102.84,82.47,89.67,91.87
1,country/region,Australia,transit,NaN,NaN,100,101.78,100.64,99.58,98.34,...,38.74,40.95,43.1,44.88,46.07,48.42,48.99,39.3,45.73,46.56
2,country/region,Australia,walking,NaN,NaN,100,101.31,101.82,104.52,113.73,...,73.27,64.36,67.05,69.46,70.26,75.89,89.6,73.16,69.72,70.49
